In [1]:
# make the notebook more readable:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import os
import xarray as xr
from intake import open_catalog
import numpy as np

cat_address = "https://raw.githubusercontent.com/observingClouds/eurec4a-intake/simulations/catalog.yml"
cat = open_catalog(cat_address)

In [24]:
# there is no radiation for domain 2 in the catalog
for item in [*list(cat.simulations.ICON.experiment_1), *list(cat.simulations.ICON.experiment_2)]: 
    if 'radi' in item: print(item)

radiation_DOM01


In [26]:
exp1_sfc_DOM01 = cat.simulations.ICON.experiment_1.surface_DOM01.to_dask()
exp1_rad_DOM01 = cat.simulations.ICON.experiment_1.radiation_DOM01.to_dask()
exp1_sfc_DOM01 = exp1_sfc_DOM01.rename({'height':'height_0'}) # both have different 'height's wich produces merge conflicts 

exp1_DOM01 = xr.merge([exp1_sfc_DOM01, exp1_rad_DOM01])
exp1_DOM01

<xarray.Dataset>
Dimensions:         (time: 10217, cell: 4528560, height_2: 1, height_0: 1,
                     height: 68, bnds: 2)
Coordinates:
  * time            (time) datetime64[ns] 2020-01-09T10:00:00 ... 2020-02-14T...
Dimensions without coordinates: cell, height_2, height_0, height, bnds
Data variables: (12/22)
    clct            (time, cell) float32 dask.array<chunksize=(12, 262144), meta=np.ndarray>
    lhfl_s          (time, cell) float32 dask.array<chunksize=(12, 262144), meta=np.ndarray>
    pres_sfc        (time, cell) float32 dask.array<chunksize=(12, 262144), meta=np.ndarray>
    qv_2m           (time, height_2, cell) float32 dask.array<chunksize=(12, 1, 262144), meta=np.ndarray>
    rain_gsp_rate   (time, cell) float32 dask.array<chunksize=(12, 262144), meta=np.ndarray>
    rh_2m           (time, height_2, cell) float32 dask.array<chunksize=(12, 1, 262144), meta=np.ndarray>
    ...              ...
    ddt_temp_radsw  (time, height, cell) float32 dask.array<chunksize=(8, 4, 262144), meta=np.ndarray>
    height_bnds     (time, height, bnds) float64 dask.array<chunksize=(8, 4, 2), meta=np.ndarray>
    sob_t           (time, cell) float32 dask.array<chunksize=(8, 262144), meta=np.ndarray>
    sod_t           (time, cell) float32 dask.array<chunksize=(8, 262144), meta=np.ndarray>
    sou_t           (time, cell) float32 dask.array<chunksize=(8, 262144), meta=np.ndarray>
    thb_t           (time, cell) float32 dask.array<chunksize=(8, 262144), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Hauke Schulz (m300408) on m21732 (Linux 2.6.32-754....
    history:              /mnt/lustre02/work/mh0010/m300408/Simulations/ICON/...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  99
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@ba5ebcf0ebc106...
    title:                ICON simulation
    uuidOfHGrid:          6b59890b-99f3-939b-e76a-0a3ad2e43140

In [28]:
keeplist = ['clct', 'sob_t']
# i just noticed, that there is no longwave radiation ...
droplist = list(exp1_DOM01)
for item in keeplist: droplist.remove(item)
data = exp1_DOM01.drop_vars(droplist)
data

<xarray.Dataset>
Dimensions:  (time: 10217, cell: 4528560)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-09T10:00:00 ... 2020-02-14T02:55:00
Dimensions without coordinates: cell
Data variables:
    clct     (time, cell) float32 dask.array<chunksize=(12, 262144), meta=np.ndarray>
    sob_t    (time, cell) float32 dask.array<chunksize=(8, 262144), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Hauke Schulz (m300408) on m21732 (Linux 2.6.32-754....
    history:              /mnt/lustre02/work/mh0010/m300408/Simulations/ICON/...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  99
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@ba5ebcf0ebc106...
    title:                ICON simulation
    uuidOfHGrid:          6b59890b-99f3-939b-e76a-0a3ad2e43140

In [5]:
# the intake catalog seems to be missing longwave radiation.
# it exists in the netCDF files:

def basename(f):
    return f.split('/')[-1].split('.')[0]
def get_timestr(f):
    return basename(f).split('_')[-1]

# get all surface data
dirname = '/work/mh0010/m300408/DVC-test/EUREC4A-ICON/EUREC4A/experiments/EUREC4A/surface/'
sur_list = [os.path.join(dirname, f) for f in os.listdir(dirname) if '_DOM01_' in f]
sur_list.sort()

# get all the radiation data
dirname = '/work/mh0010/m300408/DVC-test/EUREC4A-ICON/EUREC4A/experiments/EUREC4A/radiation/'
rad_list = [os.path.join(dirname, f) for f in os.listdir(dirname) if '_DOM01_' in f]
rad_list.sort()

# select only the ones which correspond to file in radiation data
# (surface gets written out with more frequency)
match_list = [f for f in sur_list if get_timestr(f) in [get_timestr(rad) for rad in rad_list] ]
match_list.sort()
sur_list = match_list

In [32]:
# there are some doubl time steps in this data. i dont know how to deal with that.
work_slice = slice(0,10)
rad = xr.open_mfdataset(rad_list[work_slice])
sur = xr.open_mfdataset(sur_list[work_slice])
data = xr.merge([sur,rad])
data

<xarray.Dataset>
Dimensions:         (height: 69, height_2: 69, time: 48, ncells: 4528560,
                     bnds: 2)
Coordinates:
  * height          (height) float64 10.0 83.0 84.0 85.0 ... 148.0 149.0 150.0
  * height_2        (height_2) float64 2.0 83.0 84.0 85.0 ... 148.0 149.0 150.0
  * time            (time) float64 2.02e+07 2.02e+07 ... 2.02e+07 2.02e+07
Dimensions without coordinates: ncells, bnds
Data variables: (12/26)
    u_10m           (time, height, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    v_10m           (time, height, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    rh_2m           (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    t_2m            (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    qv_2m           (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    t_seasfc        (time, ncells) float32 dask.array<chunksize=(6, 4528560), meta=np.ndarray>
    ...              ...
    ddt_temp_radsw  (time, height, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    ddt_temp_radlw  (time, height, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    lwflx_dn_clr    (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    lwflx_dn        (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    lwflx_up_clr    (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
    lwflx_up        (time, height_2, ncells) float32 dask.array<chunksize=(6, 69, 4528560), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    number_of_grid_used:  99
    uuidOfHGrid:          6b59890b-99f3-939b-e76a-0a3ad2e43140
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               icon-nwp\tgit@gitlab.dkrz.de:icon/icon-nwp.git@7e23...
    history:              /work/mh0010/m300408/DVC-test/icon-aes/bin/icon at ...
    references:           see MPIM/DWD publications
    comment:              Hauke Schulz (m300408) on l30466 (Linux 4.18.0-305....

In [33]:
keeplist = ['clct', 'sob_t', 'thb_t']
droplist = list(data)
for item in keeplist: droplist.remove(item)

data = data.drop_vars(droplist)
data

<xarray.Dataset>
Dimensions:   (height: 69, height_2: 69, time: 48, ncells: 4528560)
Coordinates:
  * height    (height) float64 10.0 83.0 84.0 85.0 ... 147.0 148.0 149.0 150.0
  * height_2  (height_2) float64 2.0 83.0 84.0 85.0 ... 147.0 148.0 149.0 150.0
  * time      (time) float64 2.02e+07 2.02e+07 2.02e+07 ... 2.02e+07 2.02e+07
Dimensions without coordinates: ncells
Data variables:
    clct      (time, ncells) float32 dask.array<chunksize=(6, 4528560), meta=np.ndarray>
    sob_t     (time, ncells) float32 dask.array<chunksize=(6, 4528560), meta=np.ndarray>
    thb_t     (time, ncells) float32 dask.array<chunksize=(6, 4528560), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    number_of_grid_used:  99
    uuidOfHGrid:          6b59890b-99f3-939b-e76a-0a3ad2e43140
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    title:                ICON simulation
    source:               icon-nwp\tgit@gitlab.dkrz.de:icon/icon-nwp.git@7e23...
    history:              /work/mh0010/m300408/DVC-test/icon-aes/bin/icon at ...
    references:           see MPIM/DWD publications
    comment:              Hauke Schulz (m300408) on l30466 (Linux 4.18.0-305....

In [34]:
klout = data.where(data['clct'] == 1.0)
print(np.min(klout.clct.values), np.max(klout.clct.values), np.median(klout.clct.values))

nan nan nan


In [37]:
clear = data.where(data['clct'] == 0.0)
print(np.min(klout.clct.values), np.max(klout.clct.values), np.median(klout.clct.values))

nan nan nan


In [43]:
import emojis
print(emojis.encode(':think:'))

:think:


In [47]:
import emoji
print(emoji.emojize(':think:'))

:think:


In [48]:
emoji.list

AttributeError: module 'emoji' has no attribute 'list'